In [1]:
import os
if not os.path.exists('braindecode_lazy'):
    !git clone https://github.com/gemeinl/braindecode_lazy
    import subprocess
    subprocess.check_output('cd braindecode_lazy; git reset --hard c785237e03f6cb0d10a3d68690a6d7111b90e994; exit 0', stderr=subprocess.STDOUT, shell=True) 
    # Ręcznie trzeba zmienić nazwę:
    with open('braindecode_lazy/braindecode_lazy/datasets/tuh.py', 'r') as f:
        p = f.read()
    p = p.replace('read_all_file_names', '_read_all_file_names')
    with open('braindecode_lazy/braindecode_lazy/datasets/tuh.py', 'w') as f:
        f.write(p)

In [2]:
import numpy as np
import torch 
import time
import pandas as pd

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import matthews_corrcoef

import sys
sys.path.insert(1, "braindecode_lazy/")
from braindecode_lazy.datasets.tuh import Tuh
from braindecode_lazy.experiments.monitors_lazy_loading import compute_preds_per_trial
from braindecode.torch_ext.util import np_to_var, var_to_np
from braindecode.datautil.iterators import CropsFromTrialsIterator
from braindecode.experiments.experiment import Experiment

from pyriemann.tangentspace import TangentSpace

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/dmj/fizmed/mpoziomska/.pyenv/versions/3.6.10/envs/HW1/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda', index=0)

Dwa pliki były za ciężkie dla githuba:

In [3]:
if not os.path.exists('data/eval'):
    if not os.path.exists('data/large_files.zip'):
        import gdown
        url = 'https://www.fuw.edu.pl/~mpoziomska/data/large_files.zip'
        output = 'data/'
        gdown.download(url, output, quiet=False)
    from zipfile import ZipFile
    with ZipFile('data/large_files.zip', 'r') as zipObj:
        zipObj.extractall('data')
    


# Model 1 - sieć Shallow
Nie zdążyłam uporządkować kodów, więc na osobnym repozytorium znajduje się kod do trenowania tego modelu: https://github.com/mpoziomska/Explain_pathological_eeg/blob/master/run_model.py. Tutaj mam wgrany jedynie gotowy model i przetestuję go na małym zbiorze ewaluacyjnym.

In [4]:
t0 = time.time()
test_subset = Tuh(f"data/eval/", target='pathological')
print((time.time() - t0) / 60)

0.12043353716532389


In [5]:
batch_size = 10
input_time_length = 6000
seed = 1
n_chans = 21
remember_best_column = "valid_misclass"
model = torch.load('model_1.pt')
test_input = np_to_var(np.ones((2, n_chans, input_time_length, 1), dtype='float32')).to(device)
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]

exp = Experiment(
        model=model,
        train_set=None,
        valid_set=None,
        test_set=test_subset,
        iterator=CropsFromTrialsIterator(batch_size, input_time_length, n_preds_per_input, seed),
        loss_function=None,
        optimizer=None,
        model_constraint=None,
        monitors=None,
        stop_criterion=None,
        remember_best_column=remember_best_column,
        run_after_early_stop=False,
        batch_modifier=None,
        do_early_stop=False,
        reset_after_second_run=False
    )

In [6]:
def make_final_predictions(exp, device):
    exp.model.eval()
    setname = 'test'
    dataset = exp.datasets[setname]
    preds_per_batch = [var_to_np(exp.model(np_to_var(b[0]).to(device)))
                       for b in exp.iterator.get_batches(dataset,
                                                         shuffle=False)]
    preds_per_trial = compute_preds_per_trial(
        preds_per_batch, dataset,
        input_time_length=exp.iterator.input_time_length,
        n_stride=exp.iterator.n_preds_per_input)
    mean_preds_per_trial = [np.mean(preds, axis=(0, 2)) for preds in
                            preds_per_trial]
    mean_preds_per_trial = np.array(mean_preds_per_trial)
    return mean_preds_per_trial, dataset.y
        
mean_preds_per_trial, y = make_final_predictions(exp, device)
preds = np.argmax(mean_preds_per_trial, axis=1)
acc = np.sum(preds == y) / len(y)

mcc = matthews_corrcoef(y, preds)
print(f"Evaluation acc: {acc}, mcc: {mcc}")

Evaluation acc: 0.8214285714285714, mcc: 0.5307910421576296


# Riemannian-geometry
Model korzysta z macierzy kowariancji między kanałami EEG.

In [7]:
class TimedTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, transformer):
        self.transformer=transformer
    
    def fit(self, X, y=None):
        t = time.time()
        return_value = self.transformer.fit(X=X, y=y)
        print("{} fitting time: {:.2f}s".format(self.transformer.__repr__, time.time() - t))
        return return_value
    
    def transform(self, X):
        t = time.time()
        return_value = self.transformer.transform(X=X)
        print("{} transforming time: {:.2f}s".format(self.transformer.__repr__, time.time() - t))
        return return_value

In [8]:
def mean_acc(pred_df):
    accs = []
    for i, d in pred_df.groupby("id"):
        accs.append((d.y_pred == d.y_true).mean())
    return np.mean(accs)

In [9]:
kernel="rbf"
adaptation = False
C = 10
pipe = Pipeline([
    ("ts", TimedTransformer(TangentSpace(tsupdate=adaptation, metric="riemann"))),
    ("clf", SVC(kernel=kernel, C=C, probability=True, gamma="auto"))
])

train_X = np.load("data/mean_train_covs.npy")
train_y = np.load("data/train_pathology_labels.npy")
test_X = np.load("data/mean_eval_covs.npy")
test_y = np.load("data/eval_pathology_labels.npy")
fold_i = 1
pipe.fit(train_X, train_y)
probas_train = pipe.predict_proba(train_X)
preds_train = pipe.predict(train_X)
train_preds, test_preds, train_probas, test_probas = pd.DataFrame(),pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

column_names = ["id", "y_pred", "y_true"]
tmp = pd.DataFrame([len(train_y) * [fold_i], probas_train[:, 1], train_y], index=column_names).T
train_probas = train_probas.append(tmp, ignore_index=True)  

tmp2 = pd.DataFrame([len(train_y) * [fold_i], preds_train, train_y], index=column_names).T
train_preds = train_preds.append(tmp2, ignore_index=True)  

probas_test = pipe.predict_proba(test_X)
preds_test = pipe.predict(test_X)

tmp = pd.DataFrame([len(test_y) * [fold_i], probas_test[:, 1], test_y], index=column_names).T
test_probas = test_probas.append(tmp, ignore_index=True)  

tmp2 = pd.DataFrame([len(test_y) * [fold_i], preds_test, test_y], index=column_names).T
test_preds = test_preds.append(tmp2, ignore_index=True)  

print(f"ACC dla zbioru treningowego: {mean_acc(train_preds)}, dla zbioru ewaluacyjnego: {mean_acc(test_preds)}")

<bound method BaseEstimator.__repr__ of TangentSpace(metric='riemann', tsupdate=False)> fitting time: 6.12s
<bound method BaseEstimator.__repr__ of TangentSpace(metric='riemann', tsupdate=False)> transforming time: 0.44s
<bound method BaseEstimator.__repr__ of TangentSpace(metric='riemann', tsupdate=False)> transforming time: 0.44s
<bound method BaseEstimator.__repr__ of TangentSpace(metric='riemann', tsupdate=False)> transforming time: 0.05s
<bound method BaseEstimator.__repr__ of TangentSpace(metric='riemann', tsupdate=False)> transforming time: 0.05s
ACC dla zbioru treningowego: 0.8656111929307806, dla zbioru ewaluacyjnego: 0.8586956521739131
